## Footprint_mongoengine

In [2]:
from footprint_mongoengine import connect, disconnect
from footprint_mongoengine.models import zulip, user
import re
import pandas as pd
import numpy as np
import datetime 
import time

current_time = datetime.datetime.now()
calc_time = datetime.date(current_time.year - 3, 9, 1)

connect('', '', '45.93.203.6', 27017, 'app')

channel = 'Техническая поддержка МИЭМ'

ch_id = zulip.Stream.objects(name=channel)[0]

/var/folders/28/21swhpms51l474n_ckq4ttcw90877k/T/ipykernel_80263/1495823555.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


InvalidURI: The empty string is not valid username.

In [17]:
pd.set_option("display.max_colwidth", None)

## Distribution of messages for topics

In [3]:
import os

In [4]:
os.chdir("data/chat_topics")

In [5]:
topics_files = os.listdir()
topics_files

['кабинет.csv',
 'объединение аккаунт.csv',
 'тайга.csv',
 'jitsi.csv',
 'вика доступ проблема.csv',
 'wekan.csv',
 'gitlab.csv',
 'google account.csv']

In [6]:
all_topics = []
for file in topics_files:
    col_name = file.split(".")[0]
    tmp_df = pd.read_csv(file, usecols=[col_name])
    all_topics.append([col_name, *tmp_df[col_name].to_list()]) # в конце списка подтем кластера само название кластера

### Тестируем на wekan



Для кластера wekan собираем все сообщения из каждой подтемы, входящей в данный кластер, в один csv файл. Результат: файл `qa_wekan.csv`, содержащий все сообщения из всех подтем.

In [54]:
all_messages_wekan = zulip.Message.objects(stream=ch_id, subject__in = all_topics[5][1:], timestamp__gte=calc_time) # all_topics[5] - файл про векан на имеет 5 индекс       

In [101]:
questions, answers = [], []

len_wekan = len(all_messages_wekan)

for i in range(840, len_wekan):
    question = all_messages_wekan[i]
    answer = ''
    bound2 = i + 6 if len_wekan - i > 5 else len_wekan
    
    for j in range(i + 1, bound2): # ищем ответ на вопрос в 5 следующих сообщениях в опредленной подтеме
        if all_messages_wekan[j].mentioned and all_messages_wekan[j].mentioned[0] == question.sender:  
            answer += all_messages_wekan[j].content


    if len(answer):
        questions.append(question.content)
        answers.append(answer)    

print(len(questions), len(answers))

840 841
[] User object
840 842
[<User: User object>] User object
840 843
[<User: User object>] User object
840 844
[] User object
840 845
[] User object
841 842
[<User: User object>] User object
<p>Здравствуйте, я и несколько моих коллег не могут получить доступ к доске проекта<br>
<a href="https://cabinet.miem.hse.ru/#/project/1416/">https://cabinet.miem.hse.ru/#/project/1416/</a><br>
<a href="https://wekan.miem.hse.ru/b/Wd62oC6QqM4j2Do3k/board">https://wekan.miem.hse.ru/b/Wd62oC6QqM4j2Do3k/board</a></p> <p><span class="user-mention silent" data-user-id="2452">Тимур Лукьянов</span> <a href="#narrow/stream/1091-.D0.A2.D0.B5.D1.85.D0.BD.D0.B8.D1.87.D0.B5.D1.81.D0.BA.D0.B0.D1.8F-.D0.BF.D0.BE.D0.B4.D0.B4.D0.B5.D1.80.D0.B6.D0.BA.D0.B0-.D0.9C.D0.98.D0.AD.D0.9C/topic/wekan/near/1030222">said</a>:</p>
<blockquote>
<p>Здравствуйте, я и несколько моих коллег не могут получить доступ к доске проекта<br>
<a href="https://cabinet.miem.hse.ru/#/project/1416/">https://cabinet.miem.hse.ru/#/project/1

In [102]:
tmp_df = pd.DataFrame(np.column_stack([questions, answers]), columns=["question", "answer"])
tmp_df.to_csv(f"data/qa_data/qa_{all_topics[5][0]}.csv", columns=["question", "answer"])

In [109]:
tmp_df.question = tmp_df.question.apply(remove_html_n)
tmp_df.answer = tmp_df.answer.apply(remove_html_n)
tmp_df

,question,answer
0,"Здравствуйте, я и несколько моих коллег не могут получить доступ к доске проектаhttps://cabinet.miem.hse.ru/#/project/1416/https://wekan.miem.hse.ru/b/Wd62oC6QqM4j2Do3k/board","Тимур Лукьянов said:Здравствуйте, я и несколько моих коллег не могут получить доступ к доске проектаhttps://cabinet.miem.hse.ru/#/project/1416/https://wekan.miem.hse.ru/b/Wd62oC6QqM4j2Do3k/boardЯ один из этих коллег, которые не могут получить доступ к доске.Тимур Лукьянов писал/а:Здравствуйте, я и несколько моих коллег не могут получить доступ к доске проектаhttps://cabinet.miem.hse.ru/#/project/1416/https://wekan.miem.hse.ru/b/Wd62oC6QqM4j2Do3k/boardДобрый день! Уточните, пожалуйста, список участников. Илья Бородин обозначился)"
1,"Здравствуйте, коллеги. Не получается подать заявку на проект. Все поля заполнены, вакансия (одна) описана, но при попытке отправить заявку выдает ошибку (Заявка.jpg).","Ирина Карпова писал/а:Здравствуйте, коллеги. Не получается подать заявку на проект. Все поля заполнены, вакансия (одна) описана, но при попытке отправить заявку выдает ошибку (Заявка.jpg).Добрый день! Передали информацию о проблеме разработчикам, постараемся оперативно решить. Приносим извинения за предоставленные неудобства.Ирина Карпова писал/а:Здравствуйте, коллеги. Не получается подать заявку на проект. Все поля заполнены, вакансия (одна) описана, но при попытке отправить заявку выдает ошибку (Заявка.jpg).Проверьте, пожалуйста, сейчас. Проблема должна быть устранена. Если отрасль всё также невозможно выбрать, укажите, пожалуйста, какой браузер используется."
2,"А теперь все еще хуже: даже стандартные поля ""нечем"" заполнить - нет данных Заявка2.png","Ирина Карпова писал/а:Здравствуйте, коллеги. Не получается подать заявку на проект. Все поля заполнены, вакансия (одна) описана, но при попытке отправить заявку выдает ошибку (Заявка.jpg).Добрый день! Передали информацию о проблеме разработчикам, постараемся оперативно решить. Приносим извинения за предоставленные неудобства.Ирина Карпова писал/а:Здравствуйте, коллеги. Не получается подать заявку на проект. Все поля заполнены, вакансия (одна) описана, но при попытке отправить заявку выдает ошибку (Заявка.jpg).Проверьте, пожалуйста, сейчас. Проблема должна быть устранена. Если отрасль всё также невозможно выбрать, укажите, пожалуйста, какой браузер используется."


### Проделаем аналогичные действия на всех кластерах тем

In [10]:
for k in range(len(all_topics)):
    print(f"{all_topics[k][0]} starts!")
    start_time = time.time()
    all_messages_topic = zulip.Message.objects(stream=ch_id, subject__in = all_topics[k][1:], timestamp__gte=calc_time)      
    questions, answers = [], []

    len_topic = len(all_messages_topic)

    for i in range(len_topic):
        question = all_messages_topic[i]
        answer = ''
        bound2 = i + 6 if len_topic - i > 5 else len_topic
        
        for j in range(i + 1, bound2): # ищем ответ на вопрос в 5 следующих сообщениях в опредленной подтеме

            if all_messages_topic[j].mentioned and all_messages_topic[j].mentioned[0] == question.sender:  
                answer += all_messages_topic[j].content


        if len(answer):
            questions.append(question.content)
            answers.append(answer)    


    tmp_df = pd.DataFrame(np.column_stack([questions, answers]), columns=["question", "answer"])
    tmp_df.to_csv(f"data/qa_data/qa_{all_topics[k][0]}.csv", columns=["question", "answer"], index=False)

    print(f"{all_topics[k][0]} is written to csv file! It`s spent {time.time() - start_time}")

кабинет starts!
кабинет is written to csv file! It`s spent 1110.6097769737244
объединение аккаунт starts!
объединение аккаунт is written to csv file! It`s spent 234.48374390602112
тайга starts!
тайга is written to csv file! It`s spent 2181.0100038051605
jitsi starts!
jitsi is written to csv file! It`s spent 307.3602180480957
вика доступ проблема starts!
вика доступ проблема is written to csv file! It`s spent 183.6431360244751
wekan starts!
wekan is written to csv file! It`s spent 1432.4894950389862
gitlab starts!
gitlab is written to csv file! It`s spent 1128.0322847366333
google account starts!
google account is written to csv file! It`s spent 41.76119804382324


## Preprocessing of qa_data

In [6]:
def re_html(message: str) -> str:
    html_pattern = "<(?:\"[^\"]*\"['\"]*|'[^']*'['\"]*|[^'\">])+>"
    message = re.sub(html_pattern, '', message)
    return message

#### Протестируем на wekan

In [21]:
df_wekan = pd.read_csv("./data/qa_data/qa_jitsi.csv", index_col=False)
df_wekan.drop("Unnamed: 0", axis=1, inplace=True)

In [22]:
df_wekan.question = df_wekan.question.apply(re_html)
df_wekan.answer = df_wekan.answer.apply(re_html)

df_wekan.tail()

,question,answer
83,И снова баг с выключением микро когда кто-то заходит или выходит,"@Илья Постников это может сделать человек, который вел запись. у меня таких доступов нет"
84,Мне кажется или в джитси меняется цвет фона в зависимости он времени суток?,"@Илья Постников это может сделать человек, который вел запись. у меня таких доступов нет"
85,@Валерия Немна можешь пожалуйста скинуть сегодняшнюю запись из jitsi где-то примерно с 13:10,"@Илья Постников это может сделать человек, который вел запись. у меня таких доступов нет"
86,"Меняет цвет фона, это уже давно. Ещё можно написать имя + _цвет и зафиксировать его. Например 505_black","@Виктор Минченков , добрый день! Можете, пожалуйста, проконсультировать @Ирина Карпова ?"
87,"@Техническая поддержка МИЭМ написала письмо в поддержку, но пока не прошло ответа. Как проверить не попало ли там сообщение в спам, например?","@Полина Егорова , добрый день! ваши письма попали в спам, потому что они были написаны с яндексы почты. спасибо, что сообщили об обращении. на ваш запрос ответим по почте"


In [23]:
df_wekan.to_csv("./data/qa_data/preproc/qa_preproc_jitsi.csv", index=False)

In [123]:
import nltk
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

nltk.download("stopwords", quiet=True)

def tokenize(text):
    reg = re.compile(r'\w+')
    return reg.findall(text)

In [124]:
def custom_tokenizer(message: str) -> str:
    """

    Args:
        message (str): тема из чата в zulip

    Returns:
        str: предобработанная тема 
    """
    # сводим все буквы к меньшему регистру
    tokens = tokenize(str(message).lower())
    
    # удаление стоп-слов и пунктуации
    tokens = list(set(tokens) - set(stopwords.words("russian")) - set(punctuation))
    
    # лемматизация слов
    mystem_analyzer = Mystem()
    tokens = [mystem_analyzer.lemmatize(w) for w in tokens]
    
    #удаление цифр из темы
    tokens = [re.sub("\d+", "", w[0]) for w in tokens]

    # склеиваем предобработанные токены в единую строку
    return ' '.join(tokens).strip()

In [126]:
# работает неправильно
tmp_df_no_raw = tmp_df.apply(custom_tokenizer)
tmp_df_no_raw.to_csv(f"data/qa_data/preproc/qa_{all_topics[3][0]}.csv")

In [93]:
for i in range(len(all_messages)): # -1
    if  all_messages[i].timestamp.date() >= calc_time and all_messages[i].subject in all_topics[2]:
        print(all_messages[i].content)
        questions.append(all_messages[i].content)
        # answers.append(all_messages[i + 1].content)
print(questions)        
answers = questions[1:] 
questions = questions[:-1]
print(questions)   
tmp_df = pd.concat([questions, answers], columns=["question", "answer"], ignore_index=True)
tmp_df.to_csv(f"data/qa_data/qa_{all_topics[3][0]}.csv")

,question,answer


## Tokenize messages

In [93]:
def remove_html_n(message: str) -> str:
    """
    Remove \n and HTML tags from messages
    :param message: text
    :return: clear message
    """
    message = re.sub('<.*?>', '', message)
    message = re.sub('\\n', '', message)
    return message

In [19]:
channels = ['Техническая поддержка МИЭМ']
ch_ids = []
for channel in channels:
    ch_ids.append(collection_s.find({"name": channel})[0])

In [5]:
message_fields = ['_id', 'content', 'sender', 'stream', 'subject', 'timestamp']
df = pd.DataFrame(list(collection_m.find({'stream': ch_ids[0]["_id"]})), columns=message_fields)
df['content'] = df['content'].apply(remove_html_n)
df.to_csv('./data/message_data.csv')
df.head()

,_id,content,sender,stream,subject,timestamp
0,6457d40f1d81cdbf11fc2e2b,Сейчас у нас запустили лекцию по теормеху в ви...,63c64a0a59bdfbcdef69c22c,6457a6a036a96aa80cefabf0,hello,2020-03-27 11:00:48
1,6457d40f1d81cdbf11fc2e2c,pasted image,63c64a0a59bdfbcdef69c22c,6457a6a036a96aa80cefabf0,hello,2020-03-27 11:01:00
2,6457d40f1d81cdbf11fc2e2d,"@Петр Рыбаков я звонил @Сергей Аксенов , догов...",63c64a1359bdfbcdef69c31d,6457a6a036a96aa80cefabf0,Видеочат Jitsi,2020-03-27 11:06:06
3,6457d40f1d81cdbf11fc2e2e,"Пожалуйста, не разбегайтесь",63c64a1359bdfbcdef69c31d,6457a6a036a96aa80cefabf0,Видеочат Jitsi,2020-03-27 11:06:18
4,6457d40f1d81cdbf11fc2e64,"Здравствуйте! Можете пожалуйста, создавать мне...",63c64a3c59bdfbcdef69c770,6457a6a036a96aa80cefabf0,Запрос на новый аккаунт,2020-03-27 20:19:07


In [21]:
def contains_only_latin_letters(s: str) -> bool:
    # Проверка, содержит ли слово только латинские буквы
    return bool(re.search(r"^[a-zA-Z]+$", s))

def is_emoji(s: str) -> bool:
    # Проверка, является ли слово смайликом
    return bool(re.search(r"^[!\"#$%&\'*+,-./:;<=>?@[\]^_`{|}~]*[()]+[!\"#$%&\'*+,-./:;<=>?@[\]^_`{|}~]*$", s))

def is_hashtag(s: str) -> bool:
    # Проверка, является ли слово хэштегом
    return bool(re.search(r"^#+[a-zA-Z]+$", s))

In [22]:
def custom_tokenizer(message: str) -> List[str]:
    """
    
    :param message: 
    :return: 
    """
    tw_tokenizer = TweetTokenizer()
    tokens = tw_tokenizer.tokenize(message)
    tokens = list(set(tokens) - set(stopwords.words("russian")) - set(punctuation))
    mystem_analyzer = Mystem()
    tokens = [mystem_analyzer.lemmatize(w) for w in tokens]
    tokens = [tmp[0] for tmp in tokens]
    return tokens

In [9]:
df = pd.read_csv('./data/message_data.csv')

In [ ]:
# cloud = WordCloud(mask=cake_mask, contour_width=10, contour_color='#2e3043', background_color='#272d3b',
                #   colormap='Set3', max_words=80).generate(text_all_sent)